In [1]:
!pip install tensorflow pandas scikit-learn


In [5]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('/content/mood_dataset.csv')

# Prepare the features and labels
X = df.drop(columns=['Mood_Label']).values  # All columns except the label
y = df['Mood_Label'].values  # Mood label

# Reshape X for LSTM: (samples, days, cycles)
# We have 3 days and 3 cycles per day, making a 3x3 input shape per sample
X = X.reshape(-1, 3, 3)

# One-hot encode the labels
y = to_categorical(y, num_classes=3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(3, 3), return_sequences=True))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes: happy, sad, neutral

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.01)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.3241 - loss: 1.0919 - val_accuracy: 0.5000 - val_loss: 1.0907
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5696 - loss: 1.0687 - val_accuracy: 0.5000 - val_loss: 1.0702
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5204 - loss: 1.0101 - val_accuracy: 0.6250 - val_loss: 0.8876
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6064 - loss: 0.8911 - val_accuracy: 0.6250 - val_loss: 0.6515
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6382 - loss: 0.7779 - val_accuracy: 0.6250 - val_loss: 0.6665
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6032 - loss: 0.7570 - val_accuracy: 0.6250 - val_loss: 0.6555
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6454 - loss: 0.7263 - val_accuracy: 0.6250 - val_loss: 0.6461
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6535 - loss: 0.6903 - val_accuracy: 0.8750 - val_los

In [7]:
model.save('mood_prediction_model.keras')


In [8]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the model
model = load_model('/content/mood_prediction_model.keras')

# Define a test sample
test_sample = np.array([
    [0, 1, 0],  # Day 1 cycles
    [2, 0, 1],  # Day 2 cycles
    [1, 1, 0]   # Day 3 cycles
])

# Reshape for a single sample input
test_sample = test_sample.reshape(1, 3, 3)

# Make a prediction
prediction = model.predict(test_sample)
predicted_mood = np.argmax(prediction, axis=1)[0]

# Map to mood labels
mood_labels = {0: "Happy", 1: "Sad", 2: "Neutral"}
predicted_mood_label = mood_labels[predicted_mood]

print(f"Predicted Mood: {predicted_mood_label}")


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
Predicted Mood: Happy
